<a href="https://colab.research.google.com/github/imrogerjiang/market_analysis/blob/main/Market_Analysis_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import glob
import os
import re
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.ticker import FuncFormatter, MaxNLocator
import numpy as np
from datetime import datetime
from typing import Dict, Optional, List
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Listings

In [37]:
# ---------- Constants ----------
YEAR_MIN, YEAR_MAX = 1980, 2035
ORDER: List[str] = ['href', 'year_make_model', 'trim', "listed_price", 'transmission', 'odometer', 'seller_type']

YEAR_RE  = r'\b(19[89]\d|20[0-3]\d)\b'
PRICE_RE = r'^\s*\$\s*[\d,]+(?:\.\d{2})?\b'
ODOM_RE  = r'^\s*\d{1,3}(?:,\d{3})+\s*km\s*$'
URL_RE   = r'^(?:https?://|www\.)'
TX, SELLER = {'automatic', 'manual'}, {'private', 'dealer used'}

THRESH: Dict[str, float] = {
    'year_make_model': 0.50,
    "listed_price":           0.60,
    'transmission':    0.80,
    'odometer':        0.60,
    'seller_type':     0.70,
}

# ---------- Predicates ----------
def _ratio(mask: pd.Series) -> float:
    return float(mask.mean()) if len(mask) else 0.0

def _yr_ok(s: pd.Series) -> pd.Series:
    years = pd.to_numeric(s.astype(str).str.extract(YEAR_RE, expand=False), errors='coerce')
    return years.between(YEAR_MIN, YEAR_MAX)

PRED = {
    'year_make_model': lambda s: s.astype(str).pipe(_yr_ok) & s.astype(str).str.contains(r'[A-Za-z]', na=False),
    "listed_price":           lambda s: s.astype(str).str.match(PRICE_RE, na=False),
    'transmission':    lambda s: s.astype(str).str.strip().str.lower().isin(TX),
    'odometer':        lambda s: s.astype(str).str.match(ODOM_RE, flags=re.I, na=False),
    'seller_type':     lambda s: s.astype(str).str.strip().str.lower().isin(SELLER),
}

# ---------- Core ----------
def identify_columns(df: pd.DataFrame) -> Dict[str, Optional[str]]:
    """Identify and map each canonical column."""
    cols = list(df.columns)
    if not cols:
        return {k: None for k in ORDER}

    href_col = cols[0]

    # exclude URL-like columns from other detection
    url_ratio = {c: _ratio(df[c].astype(str).str.contains(URL_RE, case=False, na=False)) for c in cols}
    urlish = {c for c, r in url_ratio.items() if r >= 0.50}
    blocked = {href_col} | urlish

    remaining = [c for c in cols if c not in blocked]
    picks = {t: None for t in PRED}

    for t in PRED:
        if not remaining:
            break
        scores = {c: _ratio(PRED[t](df[c])) for c in remaining}
        best_col, best_score = max(scores.items(), key=lambda kv: kv[1])
        if best_score >= THRESH[t]:
            picks[t] = best_col
            remaining.remove(best_col)

    trim_col = None
    ymm = picks.get('year_make_model')
    if ymm in cols:
        i = cols.index(ymm)
        if i + 1 < len(cols):
            trim_col = cols[i + 1]

    return {'href': href_col, **picks, 'trim': trim_col}

# ---------- Cleaning ----------
def clean_and_rename_cs(df: pd.DataFrame) -> pd.DataFrame:
    """Detect, rename, clean numeric/text data, and return standardized columns."""
    mapping = identify_columns(df)
    out = pd.DataFrame()

    # Map columns
    if mapping['href'] is not None:
        out['href'] = df[mapping['href']]
    for col in ['year_make_model', 'trim', "listed_price", 'transmission', 'odometer', 'seller_type']:
        src = mapping.get(col)
        if src is not None:
            out[col] = df[src]

    # Split "year make model"
    if 'year_make_model' in out.columns:
        split_cols = out['year_make_model'].astype(str).str.split(expand=True, n=2)
        split_cols.columns = ['year', 'make', 'model']
        out = pd.concat([out, split_cols], axis=1)

    # Clean hrefs (remove query strings)
    if 'href' in out.columns:
        out['href'] = out['href'].astype(str).str.split('?').str[0]

    # Clean numeric columns
    for col in ["listed_price", 'odometer']:
        if col in out.columns:
            out[col] = (
                out[col].astype(str)
                .replace('[^\\d]', '', regex=True)
                .replace('', pd.NA)
                .astype(float)
                .astype('Int64')
            )

    # Convert odometer to thousands of km
    if 'odometer' in out.columns:
        out['odometer'] = out['odometer'] // 1000

    # Add scrape date
    out['date_scraped'] = datetime.today().date()

    # Build final tidy table
    final_cols = ['href', 'year', 'make', 'model', "listed_price", 'trim', 'odometer', 'seller_type', 'date_scraped']
    return out[[c for c in final_cols if c in out.columns]]

def clean_fb(df):
    # Split the b column into 'year', 'make', 'model'
    df[['year', 'make', 'model']] = df['b'].str.split(expand=True, n=2)



    # Rename columns
    rename_columns = {
        "x1i10hfl href": "href",
        "a": "listed_price",
        "c": "location",
        "d": "odometer",
    }

    # If marketplace scrape does not have column "c"
    if "c" not in df.columns:
      df["c"] = "NA"

    df.rename(columns=rename_columns, inplace=True)


    # removing query string
    df["href"] = df['href'].str.split('?').str[0]

    # Convert price and odometer columns to integers
    df=df[df["listed_price"]!="Free"]
    df["listed_price"] = df["listed_price"].replace(r'[^\d]', '', regex=True).astype(float).astype('Int64')
    df['odometer'] = df['odometer'].replace(r'[^\d]', '', regex=True).astype(float).astype('Int64')

    # Add a column with today's date
    df['date_scraped'] = datetime.today().date()

    # Removing listings with null values
    df = df.dropna(subset=["listed_price","odometer","year"])

    # Remove crashed listings
    df=df[df["listed_price"]!=1234]
    df=df[df["listed_price"]!=12345]

    # Select only the required columns in order
    final_columns = ['href', 'year', 'make', 'model', "listed_price", 'odometer', 'location', 'date_scraped']
    df = df[final_columns]
    return df

# Setup

In [4]:
temp = [
    # Honda Civic (AU)
    ["honda", "civic", 2006, 2011, 8],
    ["honda", "civic", 2012, 2015, 9],
    ["honda", "civic", 2016, 2021, 10],
    ["honda", "civic", 2022, 2024, 11],

    # Honda Jazz (AU)
    ["honda", "jazz", 2008, 2013, 2],
    ["honda", "jazz", 2014, 2020, 3],

    # Hyundai i30 (AU)
    ["hyundai", "i30", 2007, 2011, 1],
    ["hyundai", "i30", 2012, 2016, 2],
    ["hyundai", "i30", 2017, 2023, 3],

    # Toyota Corolla (AU)
    ["toyota", "corolla", 2007, 2012, 10],
    ["toyota", "corolla", 2013, 2018, 11],
    ["toyota", "corolla", 2019, 2024, 12],

    # Toyota Yaris (AU)
    ["toyota", "yaris", 2006, 2010, 2],
    ["toyota", "yaris", 2011, 2019, 3],
    ["toyota", "yaris", 2020, 2024, 4],

    # Mazda 2 (AU)
    ["mazda", "2", 2007, 2014, 2],
    ["mazda", "2", 2015, 2024, 3],

    # Mazda 3 (AU)
    ["mazda", "3", 2009, 2013, 2],
    ["mazda", "3", 2014, 2018, 3],
    ["mazda", "3", 2019, 2024, 4],
]

gen_lookup = pd.DataFrame(temp, columns=["make","model","year_start","year_end","gen"])
gen_lookup.to_csv(f"/content/drive/Shareddrives/market_analysis_v2/gen_lookup.csv", index=False)

In [20]:
listings = pd.DataFrame(columns=[
    "href",
    "year",
    "make",
    "model",
    "listed_price",
    "nego_price",
    "trim",
    "odometer",
    "seller_type",
    "date_scrapped",
    "location",
    "marketplace",
    "status",
])


In [33]:
# path to your folder
folder = "/content/drive/Shareddrives/data/market_analysis/*.csv"

# collect all CSV file paths
files = glob.glob(folder)

# load all into one list
dfs = []
for f in files:
    df = pd.read_csv(f)
    df=df.rename(columns={"date_scraped":"date_found"})
    dfs.append(df)

# combine everything
combined = pd.concat(dfs, ignore_index=True)

# ensure date_found is datetime
combined["date_found"] = pd.to_datetime(combined["date_found"], errors="coerce")

# sort by date_found so the latest is kept
combined = combined.sort_values("date_found")

# drop duplicates by href, keeping latest
combined = combined.drop_duplicates(subset="href", keep="last")

# if you want it back into your 'listings' variable
listings = combined.reset_index(drop=True)

# Consolidate 'price' into 'listed_price' where 'listed_price' is null
# This handles cases where some CSVs use 'price' and others 'listed_price'
if 'price' in listings.columns and 'listed_price' in listings.columns:
    listings['listed_price'] = listings['listed_price'].fillna(listings['price'])
    listings = listings.drop(columns=['price'])
elif 'price' in listings.columns and 'listed_price' not in listings.columns:
    listings = listings.rename(columns={'price': 'listed_price'})

# Apply the desired data types after loading and combining the data
listings = listings.astype({
    "href": "string",
    "year": "Int64",
    "make": "string",
    "model": "string",
    "listed_price": "Int64",
    "trim": "string",
    "odometer": "Int64",
    "seller_type": "string",
    "location": "string"
})

In [38]:
listings.to_csv(f"/content/drive/Shareddrives/market_analysis_v2/listings.csv", index=False)